In [2]:
import pandas as pd

In [4]:
df_w = pd.read_csv("all_ticks_wide.csv")
df_l = pd.read_csv("all_ticks_long.csv")
df_w["timestamp"] = pd.to_datetime(df_w["timestamp"])
df_w = df_w.sort_values(by="timestamp")

We created the function Is_Null() which takes a dataframe as input, then returns "True" if the dataframe has null values, "False" otherwise. 

In [5]:
def Is_Null(df):
    if df.isnull().astype(int).iloc[:,1].sum() > 0:
        return True
    else:
        return False

Then, we created another function Find_Outliers() which takes a dataframe and returns the outliers if the dataframe doesn't have any null values.

In [6]:
def Find_Outliers(df):
        mean = df.iloc[:,1].mean()
        std = df.iloc[:,1].std()
        upper_limit = mean + 3*std
        lower_limit = mean - 3*std
        smaller_outliers = df[df.iloc[:,1] < lower_limit]
        larger_outliers = df[df.iloc[:,1] > upper_limit]
        outliers = {"smaller_outliers" : smaller_outliers, "larger_outliers" : larger_outliers }
        
        return outliers

Latly, we created the function df_Monthly() which takes a dataframe and a key indicating the year and the month key = (year,month), and returns a new dataframe consisting of the values in the original dataframe in that certain month of the year.

In [7]:
def df_Monthly(df, key):
    df_Monthly_dictionary = {}
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df_yearly = df["timestamp"].dt.year
    for i in range(2012,2020):
        df_current_year = df.loc[df_yearly == i]
        df_current_year_monthly = df_current_year["timestamp"].dt.month
        for k in range(1,13):
             df_Monthly_dictionary[i,k] = df_current_year.loc[df_current_year_monthly == k]
    
    return df_Monthly_dictionary.get(key)

In [11]:
def Func(df,column_name,key):
    df_column = df.loc[:,["timestamp",column_name]]
    df_column_without_null = df_column.interpolate()
    df_column_monthly = df_Monthly(df_column_without_null, key)
    return Find_Outliers(df_column_monthly)

#print(Func(df_w,column_name = "CEMAS", key = (2017,2)))

for year in range(2012,2020):
      for month in range(1,13):
         print(Func(df_w,column_name = "CEMAS", key = (year,month)))

{'smaller_outliers': Empty DataFrame
Columns: [timestamp, CEMAS]
Index: [], 'larger_outliers': Empty DataFrame
Columns: [timestamp, CEMAS]
Index: []}
{'smaller_outliers': Empty DataFrame
Columns: [timestamp, CEMAS]
Index: [], 'larger_outliers': Empty DataFrame
Columns: [timestamp, CEMAS]
Index: []}
{'smaller_outliers': Empty DataFrame
Columns: [timestamp, CEMAS]
Index: [], 'larger_outliers': Empty DataFrame
Columns: [timestamp, CEMAS]
Index: []}
{'smaller_outliers': Empty DataFrame
Columns: [timestamp, CEMAS]
Index: [], 'larger_outliers': Empty DataFrame
Columns: [timestamp, CEMAS]
Index: []}
{'smaller_outliers': Empty DataFrame
Columns: [timestamp, CEMAS]
Index: [], 'larger_outliers': Empty DataFrame
Columns: [timestamp, CEMAS]
Index: []}
{'smaller_outliers': Empty DataFrame
Columns: [timestamp, CEMAS]
Index: [], 'larger_outliers': Empty DataFrame
Columns: [timestamp, CEMAS]
Index: []}
{'smaller_outliers': Empty DataFrame
Columns: [timestamp, CEMAS]
Index: [], 'larger_outliers': Empty